In [46]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import requests
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
from naver_crawler import *

In [47]:
driver = webdriver.Chrome('C:/Users/shic/Desktop/Project/chromedriver.exe')
driver.implicitly_wait(3)
# start_date='2020-01-01' end_date='2021-07-14'
pages = list(range(1,80,1)) # 2020년 1월 ~ 2021년 7월
print(pages)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]


In [48]:
# 네이버 산업분석 리포트 크롤링

idx_list = [2,3,4,5,6,10,11,12,13,14,18,19,20,21,22,26,27,28,29,30,34,35,36,37,38,42,43,44,45,46]
df = pd.DataFrame(columns = ['date','kapital','title','pdf','link','summary'])

In [39]:
url = "https://finance.naver.com/research/invest_list.nhn?&page={}".format(1)
html = req.urlopen(url)
html.encoding = 'cp949'
doc = BeautifulSoup(html,"html.parser")
srlists = doc.find_all('tr')
tr = srlists[idx_list[0]]

date = '20'+tr.find_all('td')[3].text.replace('.','')
kap = tr.find_all('td')[1].text
pdf = tr.find_all('td')[2].find('a')['href'][-17:-4]
link = tr.find('td', class_='file').find('a').attrs['href']
href = 'https://finance.naver.com/research/'+tr.find_all('td')[0].find('a')['href']
# get data from second url
html2 = urlopen(href)
sour2 = BeautifulSoup(html2.read(), "html.parser")
title = sour2.find("th",'view_sbj').contents[0].replace('\t','').replace('\n','').strip()
summary = s_temp[0].text.replace('\n','')
s_temp=sour2.find("td",'view_cnt').find_all('div')
summary = s_temp[0].text.replace('\n','')
date, kap, title, pdf, link, summary

('20210715',
 '교보증권',
 '2022년에도 성장이 지속될 업종을 중심으로 접근',
 '1626315546130',
 'https://ssl.pstatic.net/imgstock/upload/research/invest/1626315546130.pdf',
 '1. 국내 기업실적 Review- 2021년 KOSPI 영업이익 컨센서스 223.2조원, 순이익 컨센서스 184.1조원으로 전주대비 상승- 지난 주 삼성전자, LG전자, POSCO 등의 잠정실적이 발표. 여러 기업이 2분기 서프라이즈 실적을 발표하고 있으나 주가에는 전혀 반영이 되고 있지 않은 상황. 2분기를 기점으로 실적이 peak out할 것이라는 우려가 반영되고 있는 것으로 판단. 지난 한달 간 EPS 1 변화율(2021년 실적 상향) 팩터-2.74%, EPS Q1 변화율(2분기 실적 상향) 팩터 -1.75%로 부진')

In [49]:
# 네이버 투자정보 리포트 크롤링

idx_list = [2,3,4,5,6,10,11,12,13,14,18,19,20,21,22,26,27,28,29,30,34,35,36,37,38,42,43,44,45,46]
df = pd.DataFrame(columns = ['date','kapital','title','pdf','href','summary'])
count=0

for p in pages:
    
    url = "https://finance.naver.com/research/invest_list.nhn?&page={}".format(p)
    html = req.urlopen(url)
    html.encoding = 'cp949'
    doc = BeautifulSoup(html,"html.parser")
 
    srlists = doc.find_all('tr')
    
    for i in range(30):
        
        tr = srlists[idx_list[i]]

        date = '20'+tr.find_all('td')[3].text.replace('.','')
        kap = tr.find_all('td')[1].text
        pdf = tr.find_all('td')[2].find('a')['href'][-17:-4]
        link = tr.find('td', class_='file').find('a').attrs['href']
        href = 'https://finance.naver.com/research/'+tr.find_all('td')[0].find('a')['href']
        # get data from second url
        html2 = urlopen(href)
        sour2 = BeautifulSoup(html2.read(), "html.parser")
        title = sour2.find("th",'view_sbj').contents[0].replace('\t','').replace('\n','').strip()
        summary = s_temp[0].text.replace('\n','')
        s_temp=sour2.find("td",'view_cnt').find_all('div')
        summary = s_temp[0].text.replace('\n','')
    
        print(date, kap, pdf, title, href, link, summary)
        
        record  = pd.Series([date, kap, title, pdf,href,summary],index = ['date','kapital','title','pdf','href','summary'])
        
        df = df.append(record, ignore_index = True)
        
        count+=1
        if count%100==False:
            print(count)

20210715 교보증권 1626315546130 2022년에도 성장이 지속될 업종을 중심으로 접근 https://finance.naver.com/research/invest_read.nhn?nid=23473&page=1 https://ssl.pstatic.net/imgstock/upload/research/invest/1626315546130.pdf 1. 국내 기업실적 Review- 2021년 KOSPI 영업이익 컨센서스 223.2조원, 순이익 컨센서스 184.1조원으로 전주대비 상승- 지난 주 삼성전자, LG전자, POSCO 등의 잠정실적이 발표. 여러 기업이 2분기 서프라이즈 실적을 발표하고 있으나 주가에는 전혀 반영이 되고 있지 않은 상황. 2분기를 기점으로 실적이 peak out할 것이라는 우려가 반영되고 있는 것으로 판단. 지난 한달 간 EPS 1 변화율(2021년 실적 상향) 팩터-2.74%, EPS Q1 변화율(2분기 실적 상향) 팩터 -1.75%로 부진
20210715 하나금융투자 1626315089983 Global Issue & News https://finance.naver.com/research/invest_read.nhn?nid=23472&page=1 https://ssl.pstatic.net/imgstock/upload/research/invest/1626315089983.pdf ▶ 뉴욕 연은총재, 평균 인플레이션 목표제 해명- 존 윌리엄스 뉴욕 연은 총재는 시장의 우려와 달리 평균 인플레이션 목표제가 급격한 금리 인상을 가져오지 않는다고 주장- 데이터에 따르면 정책금리의 급격한 변화를 예상할 근거는 없다고 주장- 오히려 기존 인플레이션 목표제 대비 인플레이션 기대치 고정을 강화한다고 언급
20210715 유안타증권 1626313715611 변동성 높아진 원자재 수출국 통화 https://finance.naver.com/research/invest_read.nhn?nid=23471&page=1 https://ssl.pstatic.ne

20210709 하나금융투자 1625798690369 Global Issue & News https://finance.naver.com/research/invest_read.nhn?nid=23453&page=1 https://ssl.pstatic.net/imgstock/upload/research/invest/1625798690369.pdf ▶ ECB 인플레이션 목표치 2%ECB는 기존 인플레이션 목표치인 2% 바로 아래 에서 2%로 상향 수정인플레이션 목표치 상향 조정은 완화정책 기조 지속 시사또 ECB는 인플레이션 산출에 집값을 포함할 것이고 채권 매입 프로그램 등을 통해 기후 변화 대응 노력 지원하겠다고 발표 
20210709 이베스트증권 1625797852991 Qlipboard++ 주간리뷰 & 프리뷰 Vol.113 https://finance.naver.com/research/invest_read.nhn?nid=23452&page=1 https://ssl.pstatic.net/imgstock/upload/research/invest/1625797852991.pdf 연중 최저 수준에 도달한 글로벌 소재 업종 주식 상대강도 ● MSCI 선진국 소재 업종 지수는 지난 달 7% 하락하며 MSCI 선진국 지수 대비 상대강도가 올해 최저 수준을 하회● 블룸버그 원자재 지수의 산업용 금속 지수는 여전히 강한 흐름을 보이고 있지만, 글로벌 경제 전망 및 중국 수요 우려가 반영되면서 소재 업종 주식은 약세를 기록
20210709 유안타증권 1625796520140 2분기 어닝시즌을 대하는 우리의 자세 https://finance.naver.com/research/invest_read.nhn?nid=23451&page=1 https://ssl.pstatic.net/imgstock/upload/research/invest/1625796520140.pdf 1. 삼성전자의 어닝서프라이즈로 시작된 2Q 어닝시즌- 삼성전자의 잠정실적은 전망치를 크게 상회- 연초 이후 전망치의 

20210705 유안타증권 1625451602224 주도주가 바뀐 대만 증시 https://finance.naver.com/research/invest_read.nhn?nid=23435&page=2 https://ssl.pstatic.net/imgstock/upload/research/invest/1625451602224.pdf 주도주가 바뀐 대만 증시6월 대만 산업재 섹터는 +56.8% 급등. 대만 증시 내 수익률 1~10위 기업 모두 해운/운송 관련 기업지수 비중이 72.9%에 달하는 IT섹터는 오히려 -0.5% 하락. TSMC 주가 역시 약보합권(-0.3%)IT의 상대적 부진, 산업재의 강세는 한국에서도 유사하게 진행됐던 내용
20210702 미래에셋증권 1625183815417 원가 부담 완화에 따른 기업 체감경기 개선 전망 https://finance.naver.com/research/invest_read.nhn?nid=23434&page=2 https://ssl.pstatic.net/imgstock/upload/research/invest/1625183815417.pdf 원가 부담 완화에 따른 기업 체감경기 개선 전망상반기 동안 원자재 가격 상승이 기업들의 원가 부담을 높여왔으나, 정부의 가격 안정화 정책이 효과를 보이면서 기업 체감경기 개선 기대원자재 가격 안정화로 정부의 규제 압력이 완화되고, 정책 지원 여력이 확대된 가운데, 내수소비, 신에너지, 첨단산업 육성에 정책을 집중할 것으로 전망되어 해당 테마에 대한 투자 유망
20210702 하나금융투자 1625183112206 Global Issue & News https://finance.naver.com/research/invest_read.nhn?nid=23433&page=2 https://ssl.pstatic.net/imgstock/upload/research/invest/1625183112206.pdf 필라델피아 연은 총재, 연말 테이퍼링 주장패트릭 하커 필라델피아 연은 총재는 늦는 것보다

20210624 미래에셋증권 1624492684855 증시 변동성 확대와 배당 ETF https://finance.naver.com/research/invest_read.nhn?nid=23413&page=2 https://ssl.pstatic.net/imgstock/upload/research/invest/1624492684855.pdf 북미 지역 ETF 자금 유입 확대전주(6/10-6/16) 선진국 주식형 ETF로 13주 최대 규모(363.8억 달러) 자금이 순유입되었다. 북미 지역 ETF로 자금 유입이 확대된 영향이다. 특히, 미국 성장주 ETF의 자금 유입 규모가 컸는데 Vanguard Growth ETF와 Invesco QQQ Trust로 각각 340.0억 달러, 327.3억 달러가 유입되었다. 기대인플레가 하락하면서 6월 이후 나스닥지수는 전고점을 회복했고 성장주 리바운딩에 대한 기대가 자금 흐름에도 반영된 것으로 보인다
20210624 하나금융투자 1624491765621 Global Issue & News https://finance.naver.com/research/invest_read.nhn?nid=23412&page=2 https://ssl.pstatic.net/imgstock/upload/research/invest/1624491765621.pdf 애틀랜타 연은 총재, 2022년 말 첫 금리 인상 예상라파엘 보스틱 애틀랜타 연은 총재는 최근 예상치를 상회하는 지표들을 고려해 2022년 말 첫 금리 인상이 예상된다고 언급, 추가로 2023년에 기준금리가 2회 추가 인상될 것으로 예상채권 매입 프로그램에 관해서는 연준의 고용과 인플레이션 목표에 상당한 추가 진전을 이뤘다며 테이퍼링에 대해 논의해야 한다고 주장
20210624 NH투자증권 1624491069239 지속되기 어려운 원자재 가격의 차별화 https://finance.naver.com/research/invest_read.nhn?nid=23411&page=2 https://ssl.ps

20210618 유안타증권 1623973127252 친환경 투자, 축제에서 일상으로 https://finance.naver.com/research/invest_read.nhn?nid=23392&page=3 https://ssl.pstatic.net/imgstock/upload/research/invest/1623973127252.pdf 친환경 관련주의 부진 . 하지만 끝난 것은 아니다지 난 해 가 파른 상 승세를 보 였던 친 환경 관 련주들이 지 만 , 올 해는 현 저히 상 승 탄 력이 둔 화된 모 습그 러나 ESG 관 련 자 금 시 장 대 출 , 채 권 에서는 관 련된 성 격의 자 금 조 달 이 활 발하게 진 행 중수 익율의 부 진과는 별 개로 친 환경 ETF 로 의 자 금 유 입 흐 름은 속 도는 느 리지만 꾸 준히 유 입되고 있 음
신흥국 주식형 ETF에서는 -5.6억 달러로 2주 연속 자금이 유출되었다. 로존 ETF 관심 https://finance.naver.com/research/invest_read.nhn?nid=23391&page=3 https://ssl.pstatic.net/imgstock/upload/research/invest/1623896344915.pdf 신흥아시아 ETF 2주 연속 순유출  전주(6/3-6/9) 선진국 주식형 ETF로 160.0억 달러 순유입, 전주 대비 유입 규모가 확대된 반면
20210617 미래에셋증권 1623895809399 호시탐탐 https://finance.naver.com/research/invest_read.nhn?nid=23390&page=3 https://ssl.pstatic.net/imgstock/upload/research/invest/1623895809399.pdf Trend Following계량적 지표를 활용해 기업 탐방 후보군을 선정하는 자료(매주 수요일 업데이트)주가 패턴에 따라 Bottom Fishing, Trend Following 기업을 선정(애널리스트 커버, 미커버 기업으로 구분)기

...

20200220 유안타증권 1582160318626 [중국증시]China 추천종목 – China A주 https://finance.naver.com/research/invest_read.nhn?nid=21167&page=77 https://ssl.pstatic.net/imgstock/upload/research/invest/1582160318626.pdf China 추천종목 – China A중국 최대의 건설/부동산 개발 기업으로 풍부한 수주잔고, earnings coverage ratio가 3.5배 이상임. ROE가 지속적으로 증가 중이며, 8개의 국유건설기업 중 가장 높음. 현재 주가는 과거 밸류에이션 밴드 하단에 위치해 있으며, 중국 정부의 경기대응적 조치로 재평가가 이뤄질 전망.
20200220 유안타증권 1582160278416 [중국증시]China Market https://finance.naver.com/research/invest_read.nhn?nid=21166&page=77 https://ssl.pstatic.net/imgstock/upload/research/invest/1582160278416.pdf 금일 발표될 최우대금리(LPR) 인하 여부 주목- 19일 증시는 지속적인 주가 상승에 따른 피로감 및 순환매 장세 흐름을 나타내며 하락 마감- 외국인자금은 64.24억 위안 순매수(후강퉁 +34.30억 위안, 선강퉁 +29.94억 위안 순유입)- 전일까지 경제 활동 비중이 높은 광둥, 장수, 상하이 등 지역의 규모 이상 공업기업 직업복귀율 50% 이상
20200220 유안타증권 1582160233903 달러 강세에 대한 불안감 https://finance.naver.com/research/invest_read.nhn?nid=21165&page=77 https://ssl.pstatic.net/imgstock/upload/research/invest/1582160233903.pdf 달러 강세에 대한 불안감최근 달러 인덱스가 99pt 대로 복귀하면서

20200218 유안타증권 1581987128238 [중국증시]China Market https://finance.naver.com/research/invest_read.nhn?nid=21146&page=77 https://ssl.pstatic.net/imgstock/upload/research/invest/1581987128238.pdf 경기부양 기대감 고조, 증시 급등 마감- 인민은행, 1년물 MLF 입찰금리를 10bp 인하한 3.15%로 공시하며 LPR 인하에 대한 기대감 고조. 또한,1년 만기 MLF를 통해 2천억 위안 규모 유동성 투입- ‘상장사 증자 개정안’ 실시로 자금난 해소 기대감 반영된 것도 증시 큰 폭의 상승을 견인- 외국인자금은 82.79억 위안 순매수(후강퉁 +43.65억 위안, +선강퉁 39.15억 위안 순유입)
20200218 유안타증권 1581987089817 2020 총선과 세법개정이 주는 함의 https://finance.naver.com/research/invest_read.nhn?nid=21145&page=77 https://ssl.pstatic.net/imgstock/upload/research/invest/1581987089817.pdf 복지에 대한 피로감 세금과 분배 이슈경제에 대한 확실한 믿음과 성장의 지속성이 담보되지 않고 있는 현 상황에서 분배 를 강조하는 방향성에 대하여 민간 경제주체들 의 피로 가중저소득층 복지를 위한 추가 납세 의향 이 전 정권 당시보다 낮아져 있는 상황 . 2019 년의 전반적 경기상황이 악화된 만큼 좀 더 악화되었을 가능성성장과 분배에 대한 상대적 중요도를 조사한 결과에서도 성장에 비중을 두고자 하는 분위기가 지속
20200217 키움증권 1581906459420 China Weekly https://finance.naver.com/research/invest_read.nhn?nid=21144&page=77 https://ssl.pstatic.net/imgstock/upload/researc

20200214 하나금융투자 1581642978902 눈은 미국, 귀는 중국 https://finance.naver.com/research/invest_read.nhn?nid=21126&page=78 https://ssl.pstatic.net/imgstock/upload/research/invest/1581642978902.pdf KOSPI 2,200 ~ 2,250pt다음주 국내증시는 마디 지수대 KOSPI 2,200pt선 안착을 모색하는 중립수준의 주가흐름 전개를 예상. 신종 코로나바이러스(COVID-19) 창궐 직전 주가수준에 한발 더 다가서는 시장 낙폭만회 시도가 활발. 1) 3월초(3일 정협, 5일 전인대 예정) 양회를 전후한 중국 정책부양 기대, 2) 테슬라를 위시한 4차 산업혁명 신기술주가 견인하는 미국증시 역사적 신고가 랠리, 3) 중립이상의 2월 만기주 수급환경 등이 시장 코로나 패닉 상흔 극복의 핵심촉매로 기능했던 결과
20200214 유안타증권 1581641898761 [중국증시]China 추천종목 – China A주 https://finance.naver.com/research/invest_read.nhn?nid=21125&page=78 https://ssl.pstatic.net/imgstock/upload/research/invest/1581641898761.pdf China 추천종목 – China A주중국 최대의 건설/부동산 개발 기업으로 풍부한 수주잔고, earnings coverage ratio가 3.5배 이상임. ROE가 지속적으로 증가 중이며, 8개의 국유건설기업 중 가장 높음. 현재 주가는 과거 밸류에이션 밴드 하단에 위치해 있으며, 중국 정부의 경기대응적 조치로 재평가가 이뤄질 전망.
20200214 유안타증권 1581641840625 [중국증시]China Market https://finance.naver.com/research/invest_read.nhn?nid=21124&page=78 https://ssl.pstatic.

20200212 하나금융투자 1581469457629 경기안정 기대감, 뉴욕증시 상승, ETF 자산유입 https://finance.naver.com/research/invest_read.nhn?nid=21106&page=79 https://ssl.pstatic.net/imgstock/upload/research/invest/1581469457629.pdf 경기안정 기대감, 뉴욕증시 상승, ETF 자산유입최근 코로나바이러스 확산속도가 진정되는 가운데 미국 경기의 안정적 상승에 대한 기대감이 작용하며 뉴욕증시는 상승마감. 미국 경제는 고용지표 안정 속에 기업실적 개선될 것으로 전망. ETF 시장에도 6거래일 연속 자산유입
20200212 유안타증권 1581468552020 KOSPI, 급락 후 급 반등의 의미 https://finance.naver.com/research/invest_read.nhn?nid=21105&page=79 https://ssl.pstatic.net/imgstock/upload/research/invest/1581468552020.pdf KOSPI, 급락 후 급 반등의 의미KOSPI, 급락 후 급 반등 전고점까지 2.4% 남은 상황 . 이전 고점대가 저항으로 작용할 수 있지만 급락 후 빠르게 낙폭을 회복한 것으로 장기 상승 추세 유효하다는 의미KOSPI 의 이평선들이 정배열된 가운데 120 일 이평선 아래까지 떨어지고 20 일 내에서 이평선들의 저항을 모두 회복한 2007 년 초와 2016 년 9 월의 경우 단기적 흐름 은 차이가 있어도 장기 상승세가 이어졌음
20200211 키움증권 1581387171470 달러화, 안전자산 수요에 상승 https://finance.naver.com/research/invest_read.nhn?nid=21104&page=79 https://ssl.pstatic.net/imgstock/upload/research/invest/1581387171470.pdf 유로화, 지표 부진 지속되며 달러 대비 하락달러화

20200210 유안타증권 1581295808296 [중국증시]China Market https://finance.naver.com/research/invest_read.nhn?nid=21084&page=79 https://ssl.pstatic.net/imgstock/upload/research/invest/1581295808296.pdf 반등세 주춤, 7일 증시 소폭 상승 마감- 지난 주 중국 증시는 연휴 이후 첫 거래일에 대폭락했지만 저가매수세 유입, 중국 당국의 유동성 공급및 향후 부양책 기대감으로 4거래일 연속 상승세를 나타냄. 다만, 주간 상해증시는 3.38% 낙폭을 기록- 금일 6시 30분(현지시간) 기준 중국 신종코로나 사망자가 900명을 돌파하며 사스로 인한 전 세계 사망자 수(744명)를 초과. 이번 사태로 올해 중국 GDP 0% 전망 등 비관론까지 제기되며 우려가 지속적으로확산하는 가운데, 10일부터 정상근무 재개로 연휴 이후 신종 코로나 확산 추이에 지속적인 주목 필요
20200210 유안타증권 1581295765321 [중국증시]China 추천종목 – China A주 https://finance.naver.com/research/invest_read.nhn?nid=21083&page=79 https://ssl.pstatic.net/imgstock/upload/research/invest/1581295765321.pdf China 추천종목 – China A주중국 최대의 건설/부동산 개발 기업으로 풍부한 수주잔고, earnings coverage ratio가 3.5배이상임. ROE가 지속적으로 증가 중이며, 8개의 국유건설기업 중 가장 높음. 현재 주가는 과거 밸류에이션 밴드 하단에 위치해 있으며, 중국 정부의 경기대응적 조치로 재평가가 이뤄질 전망.


In [50]:
df

,date,kapital,title,pdf,href,summary
0,20210715,교보증권,2022년에도 성장이 지속될 업종을 중심으로 접근,1626315546130,https://finance.naver.com/research/invest_read...,1. 국내 기업실적 Review- 2021년 KOSPI 영업이익 컨센서스 223.2...
1,20210715,하나금융투자,Global Issue & News,1626315089983,https://finance.naver.com/research/invest_read...,"▶ 뉴욕 연은총재, 평균 인플레이션 목표제 해명- 존 윌리엄스 뉴욕 연은 총재는 시..."
2,20210715,유안타증권,변동성 높아진 원자재 수출국 통화,1626313715611,https://finance.naver.com/research/invest_read...,1. 취약성을 드러낸 원자재 신흥국 통화7월 원자재 수출국 통화가 급격한 약세를 보...
3,20210714,미래에셋증권,피크아웃 우려 없는 실적주,1626230195496,https://finance.naver.com/research/invest_read...,실적 Peak-out 우려에도 이익모멘텀 개선세는 지속- 국내 기업들의 2021년 ...
4,20210714,하나금융투자,Global Issue & News,1626228224426,https://finance.naver.com/research/invest_read...,▶ 미국 6월 CPI +5.4%- 미국 6월 CPI는 전년 동월 대비 +5.4% 상...
...,...,...,...,...,...,...
2365,20200210,하나금융투자,바닥과 반등 사이,1581297254350,https://finance.naver.com/research/invest_read...,외국인 지수선물 투매 소강전환은 베이시스 자율반등과 증권 PR 차익매수 선회로 파급...
2366,20200210,이베스트증권,박제가 되어버린 천재(天才),1581296092206,https://finance.naver.com/research/invest_read...,코로나 바이러스 현황: 방심은 금물코로나 바이러스에 대한 우려가 완화되고 금융시장이...
2367,20200210,이베스트증권,방심은 금물,1581296054944,https://finance.naver.com/research/invest_read...,"MSCI 분기 리뷰 (2/12)MSCI가 2월, 5월, 8월, 11월에 실시하는 분..."
2368,20200210,유안타증권,[중국증시]China Market,1581295808296,https://finance.naver.com/research/invest_read...,"반등세 주춤, 7일 증시 소폭 상승 마감- 지난 주 중국 증시는 연휴 이후 첫 거래..."


In [52]:
os.chdir('C:\\Users\\shic\\Desktop\\Project\\1_Data')
os.getcwd()

'C:\\Users\\shic\\Desktop\\Project\\1_Data'

In [53]:
df.to_csv('naver_finance_invest_list_summary.csv',index=False)

In [54]:
a = pd.read_csv('naver_finance_invest_list_summary.csv')
a.shape

(2370, 6)

In [55]:
a.head()

,date,kapital,title,pdf,href,summary
0,20210715,교보증권,2022년에도 성장이 지속될 업종을 중심으로 접근,1626315546130,https://finance.naver.com/research/invest_read...,1. 국내 기업실적 Review- 2021년 KOSPI 영업이익 컨센서스 223.2...
1,20210715,하나금융투자,Global Issue & News,1626315089983,https://finance.naver.com/research/invest_read...,"▶ 뉴욕 연은총재, 평균 인플레이션 목표제 해명- 존 윌리엄스 뉴욕 연은 총재는 시..."
2,20210715,유안타증권,변동성 높아진 원자재 수출국 통화,1626313715611,https://finance.naver.com/research/invest_read...,1. 취약성을 드러낸 원자재 신흥국 통화7월 원자재 수출국 통화가 급격한 약세를 보...
3,20210714,미래에셋증권,피크아웃 우려 없는 실적주,1626230195496,https://finance.naver.com/research/invest_read...,실적 Peak-out 우려에도 이익모멘텀 개선세는 지속- 국내 기업들의 2021년 ...
4,20210714,하나금융투자,Global Issue & News,1626228224426,https://finance.naver.com/research/invest_read...,▶ 미국 6월 CPI +5.4%- 미국 6월 CPI는 전년 동월 대비 +5.4% 상...
